In [1]:
#Import all necessary modules
import os
import sys
import json

In [2]:
#Reads x, y, z coordinates from the file
def read_coordinates(filename): #specify the full path of the .coord file to be read
    coordinate_list = []
    fulllist = []
    with open(filename,"r") as f:
        line_list = f.readlines()
        for line in line_list:
            if (line.isspace()): #x, y and z coordinate arrays are seperated by blank space between rows
                fulllist.append(coordinate_list)
                coordinate_list = []
            else:
                for coord_i in line.split():
                    coordinate_list.append(float(coord_i))
        if (f.read() == ''): #indicates that we have reached the end of the file
            fulllist.append(coordinate_list)
    return fulllist

In [3]:
#Finds the units in which coordinate grid is defined
def read_distance_units(filename): #specify the full path of the grid_x/y/z.dat file to be read
    with open(filename,"r") as f:
        line_list = f.readlines()
        for word in line_list[0].split():
            if (word[:8] == 'Position'): #the distance unit is mentioned as Position(unit) in .dat file
                unit = word[-3:-1]
    return unit

In [4]:
#Reads 3D Potential data
def read_potential(filename): #specify the full path of the .dat file to be read
    potential = []
    with open(filename,"r") as f:
        line_list = f.readlines()
        for line in line_list:
            for coord_i in line.split():
                potential.append(coord_i)
    return potential

In [5]:
#Reads control values, control names and voltage units for a run from .log file 
def read_logfile(filename, search_phrase = 'STARTING CALCULATION FOR BIAS POINT'): #specify the full path of the .log file to be read
    with open(filename,"r") as f:
        line_list = f.readlines()
        for i, line in enumerate(line_list):
            if (search_phrase in line):
                dump_in = [word for word in line_list[i + 1].split()]
                #the voltage for each gate is in the form ctrl_name, ctrl_value, unit 
                control_names = [(dump_in[i]) for i in range (1, len(dump_in), 3)]
                gate_voltages = [float(dump_in[i]) for i in range (2, len(dump_in), 3)]
                voltage_unit = [(dump_in[i]) for i in range (3, len(dump_in), 3)]
                if (len(set(voltage_unit)) == 1):  #checks if voltages in all gates are given in same units or not
                    voltage_unit = voltage_unit[0]
    return tuple(gate_voltages), control_names, voltage_unit

In [6]:
#The user only needs to use this function, the functions defined above are the helper functions for SimulationReader
#Returns a dictionary that contains control names, control values, coordinates, potential data, distance units and potential units
def load_nextnano_data(directory): #specify the full path of the  directory which contains the simulation files
    datadictionary = {} 
    potential_data_gate_voltages_run = {}
    gatevoltages = []
    for folders in os.scandir(directory):
        next_directory = os.path.join(directory, folders.name)
        for logfile in (os.listdir(next_directory)):
            if (logfile[-3:] == 'log'):
                logfile_path = os.path.join(next_directory, logfile)
                gate_voltages, control_names, voltage_unit = read_logfile(logfile_path)
                gatevoltages.append(gate_voltages)
        if ('output' in logfile):
            next_next_directory = os.path.join(next_directory, 'output')
            filename = os.path.join(next_next_directory, 'grid_x.dat') #assuming x, y, z coordinates are defined in same units
            if ((os.path.isfile(filename))):
                distance_units = read_distance_units(filename)
            for sub_subfolders in os.scandir(next_next_directory):
                if (sub_subfolders.name[:4] == 'bias' and sub_subfolders.name[-3:] != 'log'):
                    next_next_next_directory = os.path.join(next_next_directory, sub_subfolders.name)
                    for datafiles in os.listdir(next_next_next_directory):
                        if (datafiles == 'potential.coord'):
                            datafiles_path = os.path.join(next_next_next_directory, datafiles)
                            coordinates = read_coordinates(datafiles_path) 
                        if (datafiles == 'potential.dat'):
                            datafiles_path = os.path.join(next_next_next_directory, datafiles)
                            potential = read_potential(datafiles_path)
                            potential_data_gate_voltages_run[str(gate_voltages)] = potential
    #the coordinate grid and control names are same for all runs within a simulation, so store them only once
    datadictionary = {'control_names' : control_names, 'control_values' : gatevoltages, 'coordinates' : coordinates, 'potential_data' : potential_data_gate_voltages_run, 'distance_units' : distance_units, 'voltage_units' : voltage_unit}
    #DataDictionary is in a format suitable to be directly converted to a .json file, its a regular python dictionary
    return datadictionary

In [7]:
#saves the dictionary containing nextnano data returned by load_nextnano_data to a json file into a user specified directory destination_folder, under the name json_filename
def save_nextnano_data(directory, json_filename, destination_folder, overwrite = False, save_version = False):
    datadictionary = load_nextnano_data(directory)
    if json_filename in os.listdir(destination_folder):
        #display error message if a file with the same name already exists in the specified folder 
        if not overwrite:
            print('Warning, a file with the same name already exists in the directory')
        #if the user explicitly prefers to overwrite the old file with the same name, previous file is replaced with the new file
        else:
            filepath = os.path.join(destination_folder, json_filename)         
            with open(filepath, 'w') as json_file:
                json.dump(datadictionary, json_file)
    else:
        filepath = os.path.join(destination_folder, json_filename)         
        with open(filepath, 'w') as json_file:
            json.dump(datadictionary, json_file)

In [8]:
#Loads the saved json files containing nextnano data from the folder specified by the user
def process_nextnano_data(json_filename, destination_folder):
    filepath = os.path.join(destination_folder, json_filename)
    #checks if a file with the specified name exists in the folder destination_folder and loads it
    if json_filename in os.listdir(destination_folder):
        open_json = open(filepath)
        json_file = json.load(open_json)
        return json_file
    #raise an error message if file does not exist in the destination_folder
    else:
        print('No such file exists in the directory : ', destination_folder)